# Title
[]()

In [72]:

import pandas as pd
import sys
import os
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from file_functions import *
# import time
import re

# from response_processing import *
# from article_processing import create_text_dict_from_folder
# import traceback
from file_functions import *

from IPython import display
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load scraped article_dict

In [94]:
path = '../web_articles/2023-06-21'
article_dict = loadpickle('scraped_articles_dict_2023-06-21_2136.sav', path)

	Time completed: 2023-06-22 18:45:46.211602
Dictionary keys: [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3, 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4, 3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.1, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16, 3.17, 3.18, 3.19, 3.2, 3.21, 3.22, 3.23, 3.24, 3.25, 3.26, 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 3.27, 3.28, 3.29, 0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 2.0, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 2.07, 2.08, 2.09]


## test_article_dict

In [47]:
test_article_dict = {
    1.11: article_dict[1.11], 
    1.12: article_dict[1.12]
    }
test_article_dict.keys()

dict_keys([1.11, 1.12])

## `partial_article_dict` function

In [21]:
def create_partial_article_dict(article_dict, n_articles=2, journals='all'):
    """
    Creates a partial article dictionary from the full article dictionary.
    
    Args:
        article_dict (dict): The full article dictionary.
        n_articles (int, optional): The number of articles per journal to include in the partial dictionary.
            Defaults to 2.
        journals ('all', int, or list, optional): The integers of the journals to include in the partial dictionary.
            Defaults to 'all'.
    
    Returns:
        dict: A partial article dictionary.
    """
    if journals == 'all':
        journals = list(set([key//1 for key in article_dict.keys()]))
    elif (type(journals) == float) or (type(journals) == int):
        journals = [journals]
    article_dict = {
        key: article_dict[key] for key in article_dict.keys() if \
        (key//1 in journals) and (key - int(key) < n_articles/100)
        }
    print(f'Keys for article_dict: {[key for key in sorted(article_dict.keys())]}')
    journals = [journal for journal in set([key["journal"] for key in article_dict.values()])]
    print('Journals:')
    for journal in journals:
        print(f'\t{journal}')
    return article_dict

partial_article_dict = dict()
test_id = 1
partial_article_dict[test_id] = create_partial_article_dict(article_dict, n_articles=2, journals='all')

Keys for article_dict: [0.0, 0.01, 1.0, 1.01, 2.0, 2.01, 3.0, 3.01]
Journals:
	BMJ Open
	Journal of Medical Internet Research
	PLOS Medicine
	PLOS One


In [22]:

test_id = 2
partial_article_dict[test_id] = create_partial_article_dict(article_dict, n_articles=2, journals=2)

Keys for article_dict: [2.0, 2.01]
Journals:
	Journal of Medical Internet Research


# Iteration 1 trim_text

In [22]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'An error occurred on line {lineno} in {filename}: {error}')    
        print('Unable to parse article text')
        body = '<Error parsing article text>' 
        article_display = body
        abs_display = abstract
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=2, to_display=0.01,
        article_regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str='.*(<h\d>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    article_regex_str = article_regex_str.replace('\d', f'{header}')
    abs_regex_str = abs_regex_str.replace('\d', f'{header}')
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict


text_dict, display_dict = text_dict_from_web(test_article_dict, to_display='all')
display_dict[1.11]['abstract']



Regex patterns: 
	.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*
	.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*
text_dict keys: [1.11, 1.12]


## article body 1.11

In [23]:
display_dict[1.11]['body']

### display all abstracts

In [33]:
for text in text_dict:
    display.display(display_dict[text]['abstract'])

## Display abstracts from all journals

In [79]:
article_key = 0.0
# article_key - -int(-article_key // 1) 
(article_key +1) - (article_key +1) //1

0.0

In [86]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=2, to_display=0.01,
        article_regex_str='.*<h\d>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str='.*(<h\d>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    article_regex_str = article_regex_str.replace('\d', f'{header}')
    abs_regex_str = abs_regex_str.replace('\d', f'{header}')
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    for text in display_dict:
        print('Start')
        display.display(display_dict[text][type])


text_dict, display_dict = text_dict_from_web(partial_article_dict[test_id], to_display='all')

display_html(display_dict, type='abstract')


Regex patterns: 
	.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*
	.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*
Journal: BMJ Open 1.0
Journal: PLOS Medicine 3.0
Journal: PLOS One 0.0
Journal: Journal of Medical Internet Research 2.0
	An error occurred on line 8 in C:\Users\silvh\AppData\Local\Temp\ipykernel_18696\3034820676.py: 'NoneType' object has no attribute 'group'
		Unable to parse article text
	An error occurred on line 8 in C:\Users\silvh\AppData\Local\Temp\ipykernel_18696\3034820676.py: 'NoneType' object has no attribute 'group'
		Unable to parse article text
text_dict keys: [1.0, 1.01, 3.0, 3.01, 0.0, 0.01, 2.0, 2.01]
Start


Start


Start


Start


Start


Start


Start


Start


# Iteration 2 text_dict_from_web takes a range of haeders

In [48]:
print(partial_article_dict[test_id][2]['url'])
sample_text = partial_article_dict[test_id][2]['text']
sample_text

https://www.jmir.org/2023/1/e45582/


'\n<h2 tabindex="0" data-test="article-collection" class="h4 green-heading-underline width-fit-content">\n                    This paper is in the following\n                    <span class="collection__span">e-collection/theme issue:</span></h2>\n<p id="main-content" tabindex="0">\n                            Published on\n                            <time datetime="21.2023">21.6.2023\n                            </time>\n                            in\n                            <span data-test="issue-info"><a href="/2023/1" class="nuxt-link-active">\n                                    Vol 25\n                                    <!----> (2023)\n                                    <!----></a></span></p>\n<h2 class="info__hidden-title">\n                            Enhancing Evidence-Based Pharmacy by Comparing the Quality of Web-Based Information Sources to the EVInews Database: Randomized Controlled Trial With German Community Pharmacists\n                        </h2>\n<p tabindex

## function

In [70]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


# text_dict, display_dict = text_dict_from_web(partial_article_dict[test_id], to_display='all')
abs_regex_str='.*(<h3.*?>Abstract</h3>.*(?:Introduction.*)?)<h3.*?>Introduction</h3>.*References<.*'
display_dict = dict()
# text_dict, display_dict = text_dict_from_web(
#     partial_article_dict[test_id], abs_regex_str=abs_regex_str, article_regex_str=abs_regex_str, to_display='all')

text_dict, display_dict = text_dict_from_web(
    partial_article_dict[test_id], to_display='all', header=(2,4))

display_html(display_dict, type='abstract')


header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introduction</h[234]>.*References<.*
Journal: Journal of Medical Internet Research 2.0
text_dict keys: [2.0, 2.01]

************************************* Start *************************************


************************************* Start *************************************


### all journals

#### display abstract

In [71]:
text_dict, display_dict = text_dict_from_web(
    partial_article_dict[1], to_display='all', header=(2,4))

display_html(display_dict, type='abstract')

header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introduction</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: PLOS Medicine 3.0
Journal: PLOS One 0.0
Journal: Journal of Medical Internet Research 2.0
text_dict keys: [1.0, 1.01, 3.0, 3.01, 0.0, 0.01, 2.0, 2.01]

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


#### Display body

In [75]:
text_dict, display_dict = text_dict_from_web(
    partial_article_dict[1], to_display='all', header=(2,4))

display_html(display_dict, type='body')

header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introduction</h[234]>.*References<.*
Journal: BMJ Open 1.0
Journal: PLOS Medicine 3.0
Journal: PLOS One 0.0
Journal: Journal of Medical Internet Research 2.0
text_dict keys: [1.0, 1.01, 3.0, 3.01, 0.0, 0.01, 2.0, 2.01]

************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


# Iteration 3 Re-scrape, but keep keywords

In [77]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])


class crawler_RSS1(scrapy.Spider):
    name = "crawler_RSS1"
    
    def __init__(self, n_articles='all'):
        self.n_articles = n_articles
    
    def start_requests(self):
        journals = {
            'PLOS One': 'https://journals.plos.org/plosone/feed/atom',
            'BMJ Open': 'https://bmjopen.bmj.com/rss/current.xml',
            'Journal of Medical Internet Research': 'https://www.jmir.org/feed/atom',
            'PLOS Medicine': 'https://journals.plos.org/plosmedicine/feed/atom'

            # 'Annual Review of Medicine': 'https://www.annualreviews.org/action/showFeed?ui=45mu4&mi=3fndc3&ai=sm&jc=med&type=etoc&feed=atom' # response code 403
            }
        for index, journal in enumerate(journals):
            # article_dict[index] = dict()
            yield scrapy.Request(
                url=journals[journal], callback=self.parse_front, 
                cb_kwargs={'journal': journal, 'journal_index': index, 'article_dict': article_dict}
                )
    
    def parse_front(self, response, journal, journal_index, article_dict):
        response.selector.remove_namespaces() # This is needed for any Atom feeds
        # print('Initiation')
        try:
            if self.n_articles != 1:
                article_title = response.xpath('//entry/title/text()').getall()
                article_url = response.css('entry > link[rel="alternate"]::attr(href)').getall()
                if article_url == []:
                    print(f'\tExtracting using method 2 for {journal}')
                    article_title = response.xpath('//item/title/text()').getall()
                    article_url = response.css('item > link::text').getall()
            else:
                article_title = [response.xpath('//entry/title/text()').get()]
                article_url = [response.css('entry > link[rel="alternate"]::attr(href)').get()]
                if article_url[0] is None:
                    print(f'\tExtracting using method 2 for {journal}')
                    article_title = [response.xpath('//item/title/text()').get()]
                    article_url = [response.css('item > link::text').get()]
        except:
            print('fail')
        print(f'Found {len(article_title)} articles and {len(article_url)} URLs for {journal}')

        # This is required for BMJ Open, which for some reason repeats each article title.
        if len(article_title) == len(article_url) * 2:
            unique_article_title = []
            [unique_article_title.append(article) for article in article_title if article not in unique_article_title]
            article_title = unique_article_title
            print(f'\tCorrected number of article titles: {len(article_title)}')
        if type(self.n_articles) == int:
            article_url = article_url[:self.n_articles]

        for index, url in enumerate(article_url):
            print(url)
            key = round(journal_index + index/100, 2)
            article_dict[key] = {
                'journal': journal,
                'title': article_title[index],
                'url': url
            }
            yield response.follow(
                url=url, callback=self.parse_pages, 
                cb_kwargs={'key': key, 'article_dict': article_dict})
                
    
    def parse_pages(self, response, key, article_dict):
        # print(f'Journal #{key}')
        text = response.xpath('//h2|//p|//h3|//h4').extract()
        article_dict[key]['text'] = ''.join(['\n'+line for line in text])
        if key - int(key) == 0:
            print(f'\t{article_dict[key]["journal"]}')
            print(f'\t\tArticle attributes: {[key for key in article_dict[key].keys()]}')
        
@wait_for(40)
def run_RSS_spider(n_articles='all'):
    """
    Scrape articles from RSS feeds. Must instantiate a blank dictionary as `article_dict` before running the script.
    Parameters:
        - n_articles (int): Number of articles to scrape from each journal. 
            If 'all' or other non-integer value, scrape all articles. Default is 'all'.

    How to call the function:
    ```
    article_dict = dict()
    run_RSS_spider(n_articles)

    ```
    """
    crawler = CrawlerRunner()
    d = crawler.crawl(crawler_RSS1, n_articles)
    return d


setup()
article_dict = dict()
n_articles = 1
test_id = 3


run_RSS_spider(n_articles)
partial_article_dict[test_id] = article_dict

text_dict, display_dict = text_dict_from_web(
    partial_article_dict[test_id], to_display='all', header=(2,4))

# display_html(display_dict, type='abstract')
# next(iter(partial_article_dict[test_id].values()))['text']


	Extracting using method 2 for BMJ Open
Found 1 articles and 1 URLs for BMJ Open
http://bmjopen.bmj.com/cgi/content/short/13/6/e067412?rss=1
Found 1 articles and 1 URLs for PLOS One
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287578
Found 1 articles and 1 URLs for PLOS Medicine
https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1004257
Found 1 articles and 1 URLs for Journal of Medical Internet Research
https://www.jmir.org/2023/1/e48392/
	PLOS One
		Article attributes: ['journal', 'title', 'url', 'text']
	PLOS Medicine
		Article attributes: ['journal', 'title', 'url', 'text']
	BMJ Open
		Article attributes: ['journal', 'title', 'url', 'text']
	Journal of Medical Internet Research
		Article attributes: ['journal', 'title', 'url', 'text']
header: [234]
Regex patterns: 
	.*<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?(<h[234].*?>Introduction</h[234]>.*References)<.*
	.*(<h[234].*?>Abstract</h[234]>.*(?:Introduction.*)?)<h[234].*?>Introductio

In [79]:
display_html(display_dict, type='body')


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


************************************* Start *************************************


# View all titles

In [82]:
from IPython import display

def trim_text(text, article_regex=None, abs_regex=None):
    if article_regex==None:
        article_regex = '.*<h2>Abstract</h2>.*(?:Introduction.*)?(<h2.*?>Introduction</h2>.*References)<.*' 
        abs_regex = '.*(<h2>Abstract</h2>.*(?:Introduction.*)?)<h2.*?>Introduction</h2>.*References<.*' 
    try:
        body = re.search(article_regex, text, re.DOTALL).group(1)
        abstract = re.search(abs_regex, text, re.DOTALL).group(1)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to parse article text')
        body = text 
        abstract = text 
    try:
        article_display = display.HTML(body)
        abs_display = display.HTML(abstract)
    except Exception as error: 
        exc_type, exc_obj, tb = sys.exc_info()
        file = tb.tb_frame
        lineno = tb.tb_lineno
        filename = file.f_code.co_filename
        print(f'\tAn error occurred on line {lineno} in {filename}: {error}')    
        print('\t\tUnable to create HTML display')
        article_display = f'<p>{body}</p>'
        abs_display = f'<p>{abstract}</p>'
    processed_article = {
        'abstract': abstract,
        'body': body,
    }
    display_dict = {
        'article_display': article_display,
        'abs_display': abs_display
    }
    return processed_article, display_dict

def text_dict_from_web(article_dict, header=(2,4), to_display=0.01,
        article_regex_str=r'.*<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?(<h\d.*?>Introduction</h\d>.*References)<.*',
        abs_regex_str=r'.*(<h\d.*?>Abstract</h\d>.*(?:Introduction.*)?)<h\d.*?>Introduction</h\d>.*References<.*'
        ):
    """
    Create a text dictionary from a dictionary containing web-scraped articles.

    Parameters:
        article_dict (dict): Values of each dictionary item are a dictionary representing the data from a 
            single article: 'url', 'text', and 'title'.

    Returns:
        text_dict: Dictionary where each item is a string of the text of an article, starting with the title.
    """
    if type(header) == int:
        header = str(header) 
    else :
        header = rf"[{''.join([str(h) for h in range(header[0], header[-1]+1)])}]"
    print(rf'header: {header}')
    article_regex_str = article_regex_str.replace('\d', header)
    abs_regex_str = abs_regex_str.replace('\d', header)
    article_regex = rf'{article_regex_str}'
    abs_regex = rf'{abs_regex_str}'
    print(f'Regex patterns: \n\t{article_regex}\n\t{abs_regex}')
    text_dict = dict()
    display_dict = dict()
    if (type(to_display) == int) or (type(to_display) == float):
        to_display = [to_display] 
    for article_key in article_dict:
        if (article_key +1) - (article_key +1) //1 == 0: # if integer
            print(f'Journal: {article_dict[article_key]["journal"]} {article_key}')
        trimmed_text, display = trim_text(article_dict[article_key]['text'], article_regex, abs_regex)
        text_dict[article_key] = {
            'title': article_dict[article_key]['title'],
            'body': f"{article_dict[article_key]['title']}\n\n{trimmed_text['body']}",
            'abstract': trimmed_text['abstract'],
        }
        if (to_display == 'all') or (to_display == None) or (article_key in to_display):
            display_dict[article_key] = {
                'abstract': display['abs_display'],
                'body': display['article_display']
            }
    print(f'text_dict keys: {[key for key in text_dict.keys()]}')
    return text_dict, display_dict

def display_html(display_dict, type='abstract'):
    """
    Display the HTML from the dictionary of HTML displays.
    """
    print()
    for text in display_dict:
        print('************************************* Start *************************************')
        display.display(display_dict[text][type])

def article_titles(article_dict):
    """
    Print the titles of the articles in a dictionary of articles.
    """
    for article in sorted(article_dict):
        print(f"{article}: {article_dict[article]['title']}")
        print(f"\t{article_dict[article]['journal']} {article_dict[article]['url']}\n")

display_dict = dict()

# text_dict, display_dict = text_dict_from_web(
#     partial_article_dict[test_id], to_display='all', header=(2,4))

# display_html(display_dict, type='abstract')
article_titles(article_dict)

0.0: Impact of severe aortic stenosis on quality of life
	PLOS One https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287508

0.01: Evaluation of biochemical, physiological traits and percentage of essential oil of sickleweed (<i>Falcaria vulgaris</i>) population in different geographical and climatic regions
	PLOS One https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287480

0.02: The use of respiratory rate-oxygenation index to predict failure of high-flow nasal cannula in patients with coronavirus disease 2019-associated acute respiratory distress syndrome: A retrospective study
	PLOS One https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287432

0.03: TiO<sub>2</sub> NPs-immobilized silica granules: New insight for nano catalyst fixation for hydrogen generation and sustained wastewater treatment
	PLOS One https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0287424

0.04: COVID-19-related stigma among infected people in 

In [83]:
pd.DataFrame(article_dict).transpose()

,journal,title,url,text
1.00,BMJ Open,Recommendations for hand hygiene in community ...,http://bmjopen.bmj.com/cgi/content/short/13/6/...,"\n<p class=""logo-bmj-journals""><a href=""http:/..."
1.01,BMJ Open,Cannabis use for menopause in women aged 35 an...,http://bmjopen.bmj.com/cgi/content/short/13/6/...,"\n<p class=""logo-bmj-journals""><a href=""http:/..."
1.02,BMJ Open,Communication and engagement of community memb...,http://bmjopen.bmj.com/cgi/content/short/13/6/...,"\n<p class=""logo-bmj-journals""><a href=""http:/..."
1.03,BMJ Open,Patterns and determinants of incident cataract...,http://bmjopen.bmj.com/cgi/content/short/13/6/...,"\n<p class=""logo-bmj-journals""><a href=""http:/..."
1.04,BMJ Open,Persistent mortality and heart failure burden ...,http://bmjopen.bmj.com/cgi/content/short/13/6/...,"\n<p class=""logo-bmj-journals""><a href=""http:/..."
...,...,...,...,...
2.05,Journal of Medical Internet Research,Analysis of Fluoride-Free Content on Twitter: ...,https://www.jmir.org/2023/1/e44586/,"\n<h2 tabindex=""0"" data-test=""article-collecti..."
2.06,Journal of Medical Internet Research,Characteristics of Users and Nonusers of Sympt...,https://www.jmir.org/2023/1/e46231/,"\n<h2 tabindex=""0"" data-test=""article-collecti..."
2.07,Journal of Medical Internet Research,Efficacy of Neck-Specific Exercise With Intern...,https://www.jmir.org/2023/1/e43888/,"\n<h2 tabindex=""0"" data-test=""article-collecti..."
2.08,Journal of Medical Internet Research,Providing Self-Led Mental Health Support Throu...,https://www.jmir.org/2023/1/e46448/,"\n<h2 tabindex=""0"" data-test=""article-collecti..."


In [85]:
pd.DataFrame(article_dict).transpose().to_csv('../web_articles/2023-06-21/article_dict_2023-06-22.csv', sep=';')

In [92]:
def html_to_string(html):
    cleaned_text = re.sub('<[^<]+?>', '', html)  # Remove HTML tags
    cleaned_text = cleaned_text.replace('\n', ' ').replace('\r', '')  # Remove line breaks
    cleaned_text = cleaned_text.replace(',', '[comma]').replace('\r', '')  # Remove line breaks
    cleaned_text = ' '.join(cleaned_text.split())  # Remove extra spaces
    return cleaned_text

def html_to_string_df(df, column='text'):
    df[column] = df[column].apply(html_to_string)
    return df

df = pd.DataFrame(article_dict).transpose()
html_to_string_df(df, column='text').to_csv('../web_articles/2023-06-21/article_dict_2023-06-22_1836.csv')

In [93]:
df.iloc[0,-1]

'Log in using your username and password Main menu Log in using your username and password You are here Article Text PDF PDF + Supplementary Material Abstract Background Hand hygiene is an important measure to prevent disease transmission. Objective To summarise current international guideline recommendations for hand hygiene in community settings and to assess to what extent they are consistent and evidence based. Eligibility criteria We included international guidelines with one or more recommendations on hand hygiene in community settings—categorised as domestic[comma] public or institutional—published by international organisations[comma] in English or French[comma] between 1 January 1990 and 15 November 2021. Data sources To identify relevant guidelines[comma] we searched the WHO Institutional Repository for Information Sharing Database[comma] Google[comma] websites of international organisations[comma] and contacted expert organisations and individuals. Charting methods Recommend

# Update `save_article_dict`: Save article_dict as CSV
If trying to export the 'text' column, the column creates extra rows due to line breaks and commas.

In [97]:
pd.DataFrame(article_dict).transpose().drop(columns=['text'])

,journal,title,url
1.00,BMJ Open,Recommendations for hand hygiene in community ...,http://bmjopen.bmj.com/cgi/content/short/13/6/...
1.01,BMJ Open,Cannabis use for menopause in women aged 35 an...,http://bmjopen.bmj.com/cgi/content/short/13/6/...
1.02,BMJ Open,Communication and engagement of community memb...,http://bmjopen.bmj.com/cgi/content/short/13/6/...
1.03,BMJ Open,Patterns and determinants of incident cataract...,http://bmjopen.bmj.com/cgi/content/short/13/6/...
1.04,BMJ Open,Persistent mortality and heart failure burden ...,http://bmjopen.bmj.com/cgi/content/short/13/6/...
...,...,...,...
2.05,Journal of Medical Internet Research,Analysis of Fluoride-Free Content on Twitter: ...,https://www.jmir.org/2023/1/e44586/
2.06,Journal of Medical Internet Research,Characteristics of Users and Nonusers of Sympt...,https://www.jmir.org/2023/1/e46231/
2.07,Journal of Medical Internet Research,Efficacy of Neck-Specific Exercise With Intern...,https://www.jmir.org/2023/1/e43888/
2.08,Journal of Medical Internet Research,Providing Self-Led Mental Health Support Throu...,https://www.jmir.org/2023/1/e46448/


In [98]:
def save_article_dict(article_dict, path, description='scraped_articles_dict', append_version=True,
    save_pickle=True, save_json=False, to_csv=False):
    """
    Save a dictionary of articles to a file. Default behaviour is to save as a pickle only.
    Parameters:
        - article_dict (dict): Dictionary of articles.
        - path (str): Path to save the file.
        - description (str): Description of the file for the filename.
        - append_version (bool): If True, append the date to the filename.
        - save_pickle (bool): If True, save the dictionary as a pickle file.
        - save_json (bool): If True, save the dictionary as a JSON file.
        - to_csv (bool): If True, convert the dictionary to a DataFrame to save as a CSV file.
    """
    if save_pickle == True:
        savepickle(article_dict, filename=f'{description}_', path=path, append_version=append_version)
    if save_json == True:
        save_to_json(article_dict, description=description, path=path, append_version=append_version)
    if to_csv == True:
        save_csv(pd.DataFrame(article_dict).transpose().drop(columns=['text']), path=path, filename=f'{description}_',
            index=True, append_version=append_version)
        
save_article_dict(article_dict, path='../web_articles/2023-06-21/',
    save_pickle=False, to_csv=True)

File saved:  ../web_articles/2023-06-21//scraped_articles_dict_2023-06-22_1848.csv
	Time completed: 2023-06-22 18:48:02.827724


selected_articles = [
    
]

# *End of Page*